# Coloring Algorithm
Final code / workflow for the coloring algorithm.

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import sys

sys.path.append('../')
from algorithm.Voxel import Voxel
from algorithm.Lattice import Lattice
from algorithm.Surroundings import Surroundings
from algorithm.SymmetryDf import SymmetryDf
from algorithm.BondPainter import Mesovoxel, BondPainter
from algorithm.Rotation import VoxelRotater

from app.design.Designer import RunDesigner
from app.visualize.Visualizer import RunVisualizer

import numpy as np
import pandas as pd
from PyQt6.QtWidgets import QApplication
from PyQt6.QtCore import QCoreApplication

## The algorithm

### Create the desired lattice structure

Saves it to a file `data/lattice2.npy` to be loaded in a separate cell. Alternatively, you can define your own numpy array in code and create the lattice that way.

In [24]:
%gui qt

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()
    
    designer = RunDesigner(app)
    input_lattice = designer.run()
    
    if input_lattice is not None:
        print(f'Lattice received.')
        np.save("data/lattice2.npy", input_lattice)
    else:
        print("No lattice received.")

Saved lattice:
[[[1 1]
  [1 1]]

 [[2 2]
  [2 2]]

 [[0 0]
  [0 0]]]

Lattice received.


### Run the algorithm

1. Creates lattice structure with voxel, bond objects.
2. Create the full surroundings matrix which is used for comparing symmetry operations on a tiled lattice structure.
3. Paint the bond with the algorithm

In [13]:
input_lattice = np.load('data/lattice2.npy')

lattice = Lattice(input_lattice)
lattice.compute_symmetries()
# surr_manager = Surroundings(lattice)
# symmetry_df = SymmetryDf(lattice, surr_manager)

# Bond painter stuff
bond_painter = BondPainter(lattice)
bond_painter.paint_lattice()

Initializing the mesovoxel...
Coloring the lattice...
Done!


In [16]:
# Lists to store combined voxel and bond information
final_df = bond_painter.lattice.final_df()
final_df

Voxel                      Bond Colors               
      ID Material Coordinates          +x -x +y -y +z -z
0      0        1   (0, 1, 2)           2  2  2  2  1  1
1      1        1   (1, 1, 2)          -2 -2 -2 -2  1  1
2      2        1   (0, 0, 2)          -2 -2 -2 -2  1  1
3      3        1   (1, 0, 2)           2  2  2  2  1  1
4      4        0   (0, 1, 1)           3  3  3  3 -1  4
5      5        0   (1, 1, 1)          -3 -3 -3 -3 -1  4
6      6        0   (0, 0, 1)          -3 -3 -3 -3 -1  4
7      7        0   (1, 0, 1)           3  3  3  3 -1  4
8      8        0   (0, 1, 0)           3  3  3  3 -4 -1
9      9        0   (1, 1, 0)          -3 -3 -3 -3 -4 -1
10    10        0   (0, 0, 0)          -3 -3 -3 -3 -4 -1
11    11        0   (1, 0, 0)           3  3  3  3 -4 -1

In [21]:
print(lattice.unique_origami())

[0, 1, 4, 5, 8, 9]


### View the final colored lattice

In [22]:
%gui qt

if __name__ == '__main__':
    if not QCoreApplication.instance():
        app = QApplication(sys.argv)
    else:
        app = QCoreApplication.instance()

    visualizeWindow = RunVisualizer(bond_painter.lattice, app)